In [1]:
import pandas as pd
import re
import numpy as np
import zipfile

In [2]:
metadata = pd.read_csv('C:/Users/gaiac/OneDrive/Desktop/foundations of cs/metaData ztbus.csv')
metadata

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500


In [3]:
metadata['busNumber'] = metadata['busNumber'].values.astype('str')
metadata['busNumber']

0       183
1       183
2       183
3       183
4       183
       ... 
1404    208
1405    208
1406    208
1407    208
1408    208
Name: busNumber, Length: 1409, dtype: object

In [4]:
metadata['startTime_iso'] = pd.to_datetime(metadata['startTime_iso'], utc=True, format='ISO8601')
metadata['endTime_iso'] = pd.to_datetime(metadata['endTime_iso'], utc=True, format='ISO8601')


In [5]:
#1. Extract all trips with busRoute 83

In [5]:
metadata[metadata['busRoute'] == '83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03 04:42:38+00:00,1583210558,2020-03-03 19:44:51+00:00,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06 04:53:23+00:00,1583470403,2020-03-06 19:44:42+00:00,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09 14:16:13+00:00,1583763373,2020-03-09 19:34:17+00:00,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10 04:50:03+00:00,1583815803,2020-03-10 19:51:25+00:00,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12 04:56:41+00:00,1583989001,2020-03-12 19:44:57+00:00,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30 04:47:53+00:00,1669783673,2022-11-30 19:50:22+00:00,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01 05:19:41+00:00,1669871981,2022-12-01 18:20:57+00:00,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02 04:47:48+00:00,1669956468,2022-12-02 19:40:01+00:00,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02+00:00,1670389982,2022-12-07 19:19:53+00:00,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


In [7]:
#2. Extract all trips where busRoute is not a number

In [6]:
metadata[pd.to_numeric(metadata['busRoute'], errors='coerce').isna()]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,1556594336,2019-04-30 08:44:20+00:00,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21+00:00,1556851821,2019-05-03 05:53:20+00:00,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10 03:16:11+00:00,1557458171,2019-05-10 18:51:37+00:00,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13 03:10:23+00:00,1557717023,2019-05-13 23:16:13+00:00,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24 02:52:47+00:00,1558666367,2019-05-24 22:35:11+00:00,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21 22:38:32+00:00,1666391912,2022-10-22 02:42:21+00:00,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22 22:34:45+00:00,1666478085,2022-10-23 02:29:59+00:00,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25 23:35:16+00:00,1669419316,2022-11-26 03:30:39+00:00,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12+00:00,1670630112,2022-12-10 03:24:28+00:00,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


In [9]:
#3. For each (busNumber, busRoute) pair, determine the number of trips

In [7]:
metadata['busRoute' ].replace('-', None, inplace = True)
metadata.groupby(['busNumber', 'busRoute']).size()

busNumber  busRoute
183        31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
dtype: int64

In [11]:
#4. For each trip, compute the ratio between the energy consumption and the average number of passengers

In [9]:
metadata['ratio_energy_passengers'] = metadata['energyConsumption']//metadata['itcs_numberOfPassengers_mean']
metadata

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,1556594336,2019-04-30 08:44:20+00:00,1556613860,77213.87,None,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500,86405000.0
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30 13:22:07+00:00,1556630527,2019-04-30 17:54:02+00:00,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500,12147474.0
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01 05:58:51+00:00,1556690331,2019-05-01 22:32:30+00:00,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500,73427940.0
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21+00:00,1556851821,2019-05-03 05:53:20+00:00,1556862800,42565.48,None,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500,167332785.0
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03 15:41:57+00:00,1556898117,2019-05-03 23:06:24+00:00,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500,26131895.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06 14:43:49+00:00,1670337829,2022-12-06 18:22:52+00:00,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500,10702152.0
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02+00:00,1670389982,2022-12-07 19:19:53+00:00,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655,53384616.0
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08 05:22:20+00:00,1670476940,2022-12-08 18:39:15+00:00,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500,47380469.0
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12+00:00,1670630112,2022-12-10 03:24:28+00:00,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500,22477525.0


In [13]:
#5. For each station (itcs_stopName), determine the average number of passengers.

In [10]:
zf = zipfile.ZipFile('C:/Users/gaiac/Downloads/Dataset/ZTBus_compressed.zip') 
zf
dfs = []

In [11]:
cols = ['itcs_stopName', 'gnss_altitude', 'status_haltBrakeIsActive', 'status_parkBrakeIsActive', 'temperature_ambient', 'odometry_vehicleSpeed','time_iso']
coltypes = {'gnss_altitude': np.float32, 'status_haltBrakeIsActive':np.float32, 'status_parkBrakeIsActive':np.float32, 'temperature_ambient':np.float32, 'odometry_vehicleSpeed':np.float32}

In [12]:
for i in metadata['name']:
    data = pd.read_csv(zf.open(f'{i}.csv'), usecols = cols)
    data['name'] = i
    dfs.append(data)
    
details = pd.concat(dfs, ignore_index=True)
details

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name
0,2019-04-30T03:18:56Z,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
1,2019-04-30T03:18:57Z,NaN,-,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
2,2019-04-30T03:18:58Z,NaN,-,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
3,2019-04-30T03:18:59Z,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
4,2019-04-30T03:19:00Z,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
...,...,...,...,...,...,...,...,...
48674457,2022-12-11T03:29:01Z,410.1916,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674458,2022-12-11T03:29:02Z,410.8726,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674459,2022-12-11T03:29:03Z,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674460,2022-12-11T03:29:04Z,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05


In [13]:
details['time_iso'] = pd.to_datetime(details['time_iso'], utc=True, format='ISO8601')

In [14]:
merged = pd.merge(details, metadata, on= 'name')
merged

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,startTime_iso,...,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers
0,2019-04-30 03:18:56+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,None,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0
1,2019-04-30 03:18:57+00:00,NaN,-,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,None,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0
2,2019-04-30 03:18:58+00:00,NaN,-,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,None,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0
3,2019-04-30 03:18:59+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,None,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0
4,2019-04-30 03:19:00+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,None,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,N2,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0
48674458,2022-12-11 03:29:02+00:00,410.8726,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,N2,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0
48674459,2022-12-11 03:29:03+00:00,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,N2,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0
48674460,2022-12-11 03:29:04+00:00,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,N2,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0


In [15]:
merged['itcs_stopName' ].replace('-', None, inplace = True)
merged['busRoute' ].replace('-', None, inplace = True)
merged['gnss_altitude' ].replace('NaN', None, inplace = True)

In [20]:
#5. For each station (itcs_stopName), determine the average number of passengers.

In [16]:

merged.groupby('itcs_stopName')['itcs_numberOfPassengers_mean'].mean()

itcs_stopName
Zürich, Albisrank                 17.481729
Zürich, Albisriederplatz          18.117399
Zürich, Altes Krematorium         17.823082
Zürich, Bahnhof Affoltern         12.219774
Zürich, Bahnhof Altstetten        16.929800
                                    ...    
Zürich, Zentrum Witikon           21.245386
Zürich, Zweiackerstrasse          22.344507
Zürich, Zwielplatz                16.469821
Zürich, Zwinglihaus               20.956227
Zürich,Kalkbreite/Bhf.Wiedikon    19.463529
Name: itcs_numberOfPassengers_mean, Length: 149, dtype: float64

In [22]:
#6. For each station, determine the buses that have stopped there at least once.

In [17]:

merged.groupby(['itcs_stopName', 'busNumber']).size()

itcs_stopName                   busNumber
Zürich, Albisrank               183           6324
                                208           6199
Zürich, Albisriederplatz        183          11218
                                208           7633
Zürich, Altes Krematorium       183          10022
                                             ...  
Zürich, Zwielplatz              208              4
Zürich, Zwinglihaus             183           2791
                                208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  183            279
                                208            256
Length: 297, dtype: int64

In [24]:
#7. For each station, determine the buses that have stopped there at least ten times.

In [18]:
grouped = merged.groupby(['itcs_stopName', 'busNumber']).size() 
grouped[grouped >= 10]   

itcs_stopName                   busNumber
Zürich, Albisrank               183           6324
                                208           6199
Zürich, Albisriederplatz        183          11218
                                208           7633
Zürich, Altes Krematorium       183          10022
                                             ...  
Zürich, Zwielplatz              183             77
Zürich, Zwinglihaus             183           2791
                                208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  183            279
                                208            256
Length: 295, dtype: int64

In [26]:
#9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.

In [19]:
encons = merged[['busRoute', 'busNumber', 'energyConsumption']].groupby(['busRoute', 'busNumber']).sum()

In [20]:
drdist = merged[['busRoute', 'busNumber', 'drivenDistance']].groupby(['busRoute', 'busNumber']).sum()
drdist

drivenDistance
busRoute busNumber                
31       183          1.217251e+11
         208          6.173676e+10
32       183          1.265581e+11
         208          1.299722e+11
33       183          1.707359e+12
         208          3.700875e+11
46       183          8.977346e+11
         208          2.273910e+11
72       183          1.361235e+12
         208          5.506043e+11
83       183          1.945102e+12
         208          1.972750e+12
N1       183          9.688459e+09
         208          5.459662e+09
N2       183          2.002573e+10
         208          2.085606e+10
N4       183          1.032185e+10
         208          7.213984e+09

In [21]:
r_n = encons['energyConsumption'] / drdist['drivenDistance']
r_n

busRoute  busNumber
31        183          5946.821484
          208          5126.065002
32        183          6237.645442
          208          5455.069681
33        183          5969.762297
          208          5665.913705
46        183          5620.391413
          208          5572.631147
72        183          5888.668844
          208          5418.458611
83        183          5835.235358
          208          5862.878592
N1        183          5997.312522
          208          5614.007947
N2        183          5718.813987
          208          5419.186724
N4        183          6180.960477
          208          6046.228153
dtype: float64

In [22]:
r_n = r_n.to_frame()

In [23]:
type(r_n)

pandas.core.frame.DataFrame

In [24]:
r_n = r_n.reset_index()

In [25]:
r_n.rename(columns = {'busRoute': 'busRoute', 'busNumber': 'busNumber', 0: 'ratio'}, inplace= True)

In [26]:
r_n

,busRoute,busNumber,ratio
0,31,183,5946.821484
1,31,208,5126.065002
2,32,183,6237.645442
3,32,208,5455.069681
4,33,183,5969.762297
5,33,208,5665.913705
6,46,183,5620.391413
7,46,208,5572.631147
8,72,183,5888.668844
9,72,208,5418.458611


In [36]:
#10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [27]:
min_ratio = r_n[['busRoute', 'ratio']].groupby('busRoute').min()
min_ratio.rename(columns = {'ratio': 'min_ratio'}, inplace= True)
min_ratio

,min_ratio
busRoute,
31,5126.065002
32,5455.069681
33,5665.913705
46,5572.631147
72,5418.458611
83,5835.235358
N1,5614.007947
N2,5419.186724
N4,6046.228153


In [28]:
max_ratio = r_n[['busRoute','ratio']].groupby('busRoute').max()
max_ratio.rename(columns = {'ratio': 'max_ratio'}, inplace= True)
max_ratio


,max_ratio
busRoute,
31,5946.821484
32,6237.645442
33,5969.762297
46,5620.391413
72,5888.668844
83,5862.878592
N1,5997.312522
N2,5718.813987
N4,6180.960477


In [29]:
difference = max_ratio['max_ratio'] - min_ratio['min_ratio']
difference

busRoute
31    820.756482
32    782.575761
33    303.848593
46     47.760265
72    470.210233
83     27.643233
N1    383.304576
N2    299.627263
N4    134.732324
dtype: float64

In [30]:
dif = [max_ratio, min_ratio, difference]
diff = pd.concat(dif, axis = 1)
diff.reset_index
diff.rename(columns = {'max_ratio': 'max_ratio', 'min_ratio': 'min_ratio', 0:'difference'}, inplace= True)
diff

,max_ratio,min_ratio,difference
busRoute,,,
31,5946.821484,5126.065002,820.756482
32,6237.645442,5455.069681,782.575761
33,5969.762297,5665.913705,303.848593
46,5620.391413,5572.631147,47.760265
72,5888.668844,5418.458611,470.210233
83,5862.878592,5835.235358,27.643233
N1,5997.312522,5614.007947,383.304576
N2,5718.813987,5419.186724,299.627263
N4,6180.960477,6046.228153,134.732324


In [41]:
#11. Find the bus maximizing the difference computed in the previous point.

In [31]:
diff.iloc[:,[2]].max()

difference    820.756482
dtype: float64

In [43]:
#12. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation.

In [32]:
merged['match'] = merged.gnss_altitude.eq(merged.gnss_altitude.shift(1))

In [45]:
merged[merged['gnss_altitude'].isna() == False]

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,startTime_iso,...,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers,match
142,2019-04-30 03:21:18+00:00,397.8000,None,3.228894,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
143,2019-04-30 03:21:19+00:00,397.8000,None,3.236604,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,True
144,2019-04-30 03:21:20+00:00,397.8000,None,3.305451,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,True
145,2019-04-30 03:21:21+00:00,397.8000,None,3.251502,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,True
146,2019-04-30 03:21:22+00:00,397.8000,None,3.293897,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48674433,2022-12-11 03:28:37+00:00,416.0540,None,0.000000,1.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674444,2022-12-11 03:28:48+00:00,413.7306,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674456,2022-12-11 03:29:00+00:00,410.1028,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False


In [46]:
merged[(merged['gnss_altitude'].isna() == False) & (merged['match'] == False)]

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,startTime_iso,...,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers,match
142,2019-04-30 03:21:18+00:00,397.8000,None,3.228894,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
148,2019-04-30 03:21:24+00:00,397.8299,None,3.793694,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
149,2019-04-30 03:21:25+00:00,400.3140,None,4.045902,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
150,2019-04-30 03:21:26+00:00,401.6892,None,4.404461,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
151,2019-04-30 03:21:27+00:00,400.4860,None,4.860109,0.0,0.0,292.15,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,478585200.0,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48674433,2022-12-11 03:28:37+00:00,416.0540,None,0.000000,1.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674444,2022-12-11 03:28:48+00:00,413.7306,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674456,2022-12-11 03:29:00+00:00,410.1028,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.000000,0.0,1.0,278.15,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,535095600.0,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False


In [33]:
def compute_variation(df, col):
    df = df.sort_index()  
    df['altitude_variation'] = df[col].sub(df[col].shift(1))
    return df['altitude_variation']

In [48]:
merged['altitude_variation'] = compute_variation(merged, 'gnss_altitude')

In [49]:
merged

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,startTime_iso,...,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers,match,altitude_variation
0,2019-04-30 03:18:56+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN
1,2019-04-30 03:18:57+00:00,NaN,None,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN
2,2019-04-30 03:18:58+00:00,NaN,None,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN
3,2019-04-30 03:18:59+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN
4,2019-04-30 03:19:00+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,5.53886,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,0.0888
48674458,2022-12-11 03:29:02+00:00,410.8726,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,0.6810
48674459,2022-12-11 03:29:03+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,NaN
48674460,2022-12-11 03:29:04+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,15.99320,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,NaN


In [50]:
#13. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation.

In [51]:
merged.groupby('name').altitude_variation.apply(lambda x: np.sum(np.abs(x)))

name
B183_2019-04-30_03-18-56_2019-04-30_08-44-20      184.7389
B183_2019-04-30_13-22-07_2019-04-30_17-54-02     3896.8300
B183_2019-05-01_05-58-51_2019-05-01_22-32-30      637.4071
B183_2019-05-03_02-50-21_2019-05-03_05-53-20     4074.4802
B183_2019-05-03_15-41-57_2019-05-03_23-06-24    13325.9491
                                                   ...    
B208_2022-12-06_14-43-49_2022-12-06_18-22-52     2300.3956
B208_2022-12-07_05-13-02_2022-12-07_19-19-53     8617.6752
B208_2022-12-08_05-22-20_2022-12-08_18-39-15     8022.7330
B208_2022-12-09_23-55-12_2022-12-10_03-24-28     2748.4226
B208_2022-12-10_23-34-46_2022-12-11_03-29-05     3247.2228
Name: altitude_variation, Length: 1409, dtype: float64

In [52]:
#14. For each month of the year, compute the average ambient temperature

In [53]:
merged['month'] = merged['startTime_iso'].dt.month
merged

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,startTime_iso,...,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio_energy_passengers,match,altitude_variation,month
0,2019-04-30 03:18:56+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN,4
1,2019-04-30 03:18:57+00:00,NaN,None,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN,4
2,2019-04-30 03:18:58+00:00,NaN,None,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN,4
3,2019-04-30 03:18:59+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN,4
4,2019-04-30 03:19:00+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56+00:00,...,0,20,0.740640,282.3780,281.15,293.15,86405000.0,False,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,0.0888,12
48674458,2022-12-11 03:29:02+00:00,410.8726,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,0.6810,12
48674459,2022-12-11 03:29:03+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,NaN,12
48674460,2022-12-11 03:29:04+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10 23:34:46+00:00,...,0,52,0.459492,276.0036,273.15,291.15,33457694.0,False,NaN,12


In [54]:
merged[['temperature_ambient', 'month']].groupby('month').mean()

,temperature_ambient
month,
1,278.815026
2,281.205427
3,283.231337
4,287.028389
5,290.189532
6,296.771650
7,297.716663
8,296.327105
9,292.442343


In [55]:
#15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

In [56]:
time_halt = merged[['status_haltBrakeIsActive', 'busNumber']].groupby('busNumber').sum() / 60

In [57]:
time_halt.reset_index()

,busNumber,status_haltBrakeIsActive
0,183,130625.533333
1,208,99035.650000


In [58]:
time_park = merged[['status_parkBrakeIsActive', 'busNumber']].groupby('busNumber').sum()  / 60

In [59]:
time_park.reset_index()

,busNumber,status_parkBrakeIsActive
0,183,53077.316667
1,208,30477.933333


In [60]:
time_halt['status_haltBrakeIsActive']/time_park['status_parkBrakeIsActive']

busNumber
183    2.461043
208    3.249421
dtype: float64

In [61]:
#16. For each pair of stops that are consecutive in some trip, compute the average speed achieved when going from the first to the second stop.

In [62]:
details

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name
0,2019-04-30 03:18:56+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
1,2019-04-30 03:18:57+00:00,NaN,-,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
2,2019-04-30 03:18:58+00:00,NaN,-,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
3,2019-04-30 03:18:59+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
4,2019-04-30 03:19:00+00:00,NaN,-,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674458,2022-12-11 03:29:02+00:00,410.8726,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674459,2022-12-11 03:29:03+00:00,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674460,2022-12-11 03:29:04+00:00,NaN,-,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05


In [63]:
details['itcs_stopName' ].replace('-', None, inplace = True)

In [64]:
details

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name
0,2019-04-30 03:18:56+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
1,2019-04-30 03:18:57+00:00,NaN,None,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
2,2019-04-30 03:18:58+00:00,NaN,None,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
3,2019-04-30 03:18:59+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
4,2019-04-30 03:19:00+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674458,2022-12-11 03:29:02+00:00,410.8726,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674459,2022-12-11 03:29:03+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05
48674460,2022-12-11 03:29:04+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05


In [65]:
name_change_mask = details['name'] != details['name'].shift(1)
nan_mask = details['itcs_stopName'].notna()
combined_mask = name_change_mask | nan_mask


In [66]:
stop_num = combined_mask.cumsum()
stop_num = stop_num.ffill().astype(int)

In [67]:
stop_num

0                1
1                1
2                1
3                1
4                1
             ...  
48674457    446147
48674458    446147
48674459    446147
48674460    446147
48674461    446147
Length: 48674462, dtype: int32

In [68]:
details['stop_num'] = stop_num
details

,time_iso,gnss_altitude,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,stop_num
0,2019-04-30 03:18:56+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1
1,2019-04-30 03:18:57+00:00,NaN,None,0.0,0.0,0.0,292.3688,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1
2,2019-04-30 03:18:58+00:00,NaN,None,0.0,0.0,0.0,292.9310,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1
3,2019-04-30 03:18:59+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1
4,2019-04-30 03:19:00+00:00,NaN,None,0.0,0.0,0.0,293.1500,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1
...,...,...,...,...,...,...,...,...,...
48674457,2022-12-11 03:29:01+00:00,410.1916,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,446147
48674458,2022-12-11 03:29:02+00:00,410.8726,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,446147
48674459,2022-12-11 03:29:03+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,446147
48674460,2022-12-11 03:29:04+00:00,NaN,None,0.0,0.0,1.0,278.1500,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,446147


In [69]:
proof = details.groupby('stop_num', as_index=False).agg({'odometry_vehicleSpeed': 'mean', 'name': 'first', 'itcs_stopName': 'first'})

In [70]:
proof

,stop_num,odometry_vehicleSpeed,name,itcs_stopName
0,1,2.311313,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,None
1,2,6.710928,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,"Zürich, Herdernstrasse"
2,3,5.433255,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,"Zürich, Hardplatz"
3,4,4.160358,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,"Zürich, Güterbahnhof"
4,5,7.033733,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,"Zürich, Bäckeranlage"
...,...,...,...,...
446142,446143,3.571780,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,"Zürich, Zehntenhausplatz"
446143,446144,5.647805,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,"Zürich, Bahnhof Affoltern"
446144,446145,6.911009,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,"Zürich, Fronwald"
446145,446146,8.297912,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,"Zürich, Unteraffoltern"


In [71]:
proof['stops_pairs'] = proof['itcs_stopName']  + '-' + proof['itcs_stopName'].shift(-1)

In [72]:
proof[['stops_pairs', 'odometry_vehicleSpeed']]

,stops_pairs,odometry_vehicleSpeed
0,NaN,2.311313
1,"Zürich, Herdernstrasse-Zürich, Hardplatz",6.710928
2,"Zürich, Hardplatz-Zürich, Güterbahnhof",5.433255
3,"Zürich, Güterbahnhof-Zürich, Bäckeranlage",4.160358
4,"Zürich, Bäckeranlage-Zürich, Militär-/Langstrasse",7.033733
...,...,...
446142,"Zürich, Zehntenhausplatz-Zürich, Bahnhof Affol...",3.571780
446143,"Zürich, Bahnhof Affoltern-Zürich, Fronwald",5.647805
446144,"Zürich, Fronwald-Zürich, Unteraffoltern",6.911009
446145,"Zürich, Unteraffoltern-Zürich, Schwandenholz",8.297912
